# kNN Hello-World
In this sample notebook we provide Hello-world style tutorials to test 3 popular kNN options: 
* Annoy https://github.com/spotify/annoy
* FAISS https://github.com/facebookresearch/faiss
* HNSW https://github.com/nmslib/hnswlib

A popular performance benchmark is documented here https://github.com/erikbern/ann-benchmarks.
Developed on `conda_python3` kernel of an ml.c5.4xlarge SageMaker notebook instance with 20Gb ML volume.


*Provided for demo purpose*

In [ ]:
# install annoy
! pip install --user annoy

In [ ]:
# install faiss
! conda install faiss-cpu -c pytorch -y

In [ ]:
# install hnswlib
! pip install --user git+https://github.com/nmslib/hnswlib#subdirectory=python_bindings

## Bring data
As support dataset we use Glove embeddings https://nlp.stanford.edu/projects/glove/. Citation:

* Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation.


In [ ]:
%%time

# We download Glove Common Crawl (840B tokens, 2.2M vocab, cased, 300d vectors, 2.03 GB download)
! wget http://nlp.stanford.edu/data/glove.840B.300d.zip

In [ ]:
%%time

! unzip glove.840B.300d.zip

In [1]:
import csv
import json

import numpy as np
import pandas as pd

print('numpy: ' + str(np.__version__))
print('pandas: ' + str(pd.__version__))

numpy: 1.14.3
pandas: 0.24.2


In [2]:
# read word embeddings
embs = pd.read_csv(
    'glove.840B.300d.txt',
    sep=" ",
    index_col=0,
    header=None,
    quoting=csv.QUOTE_NONE)

In [3]:
print(embs.shape)

(2196017, 300)


In [4]:
# create a index-word mapping so that kNN results are readable with actual words
embmap = {i:w for i, w in enumerate(embs.index)}

In [5]:
# save the id-word map to test search index later without opening the big embedding file
with open('embmap.json', 'w') as json_file:
    json.dump(embmap, json_file)

# Annoy
https://github.com/spotify/annoy

In [6]:
# if can't be found, restart the kernel after the pip install annoy above)
from annoy import AnnoyIndex

### Build an index

In [7]:
%%time

dim = embs.shape[1]  # vector dimension

t = AnnoyIndex(f=dim, metric='angular')

for i, v in enumerate(embs.values):
    t.add_item(i, v)

t.build(10) # 10 trees
t.save('test.ann')

CPU times: user 3min 14s, sys: 2.81 s, total: 3min 17s
Wall time: 3min 17s


from annoy documentation (https://github.com/spotify/annoy#full-python-api): *"More trees gives higher precision when querying. After calling `build`, no more items can be added."*

### Search

In [8]:
# (optionnally) load index
u = AnnoyIndex(f=300, metric='angular')
u.load('test.ann')

True

In [9]:
%%time

# Search (542 is the ID of word "software")
neighborsID = u.get_nns_by_item(i=542, n=10)

CPU times: user 1.46 ms, sys: 0 ns, total: 1.46 ms
Wall time: 851 µs


In [10]:
# words IDs and actual words
print(neighborsID)
print([embmap[i] for i in neighborsID])

[542, 704, 4451, 3602, 9735, 12203, 10452, 13563, 894, 1322]
['software', 'computer', 'desktop', 'computers', 'multimedia', 'scanner', 'utilities', 'downloadable', 'internet', 'mobile']


# FAISS
https://github.com/facebookresearch/faiss

In [11]:
import faiss

### Build an index

In [12]:
d = 300  # dim
nlist = 500  # number of clusters searched - parameter to tune

quantizer = faiss.IndexFlatL2(d) # build the index
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

In [13]:
%%time

# faiss needs contiguous arrays
x = np.ascontiguousarray(embs.values).astype('float32')

index.train(x)
index.add(x)

CPU times: user 34.9 s, sys: 3.11 s, total: 38 s
Wall time: 9.91 s


In [14]:
# save index
faiss.write_index(index, 'faissIVFFlat.index')

### Search

In [15]:
# (optionnally) load index
index = faiss.read_index('faissIVFFlat.index')

In [16]:
k = 10  # number of neighbors to return

# search query needs to contiguous array of arrays
# 542 is index for word "software"
input_array = np.expand_dims(embs.iloc[542].values, axis=0)
input_array = np.ascontiguousarray(input_array).astype('float32')

In [17]:
%%time 

# search
D, I = index.search(input_array, k)

CPU times: user 181 ms, sys: 19.1 ms, total: 200 ms
Wall time: 13.9 ms


In [18]:
# labels and actual words
print(I)
print([embmap[i] for i in I.tolist()[0]])

[[  542   704  1513  1318  5084 13319  5413   899 18342  1410]]
['software', 'computer', 'tool', 'tools', 'enterprise', 'automation', 'enables', 'application', 'web-based', 'applications']


## HNSWlib

In [19]:
import hnswlib

### Build an index
* https://github.com/nmslib/hnswlib
* https://github.com/nmslib/hnswlib/blob/master/ALGO_PARAMS.md

In [20]:
dim = 300
num_elements = len(embs)

In [21]:
%%time

# Declaring index
p = hnswlib.Index(space='l2', dim=dim) # possible options are l2, cosine or ip

# Initing index - the maximum number of elements should be known beforehand
p.init_index(max_elements=num_elements, ef_construction=200, M=16)

# Element insertion (can be called several times):
p.add_items(embs.values, list(range(len(embs))))

CPU times: user 1h 4min 46s, sys: 35.2 s, total: 1h 5min 21s
Wall time: 4min 12s


In [22]:
# persist
p.save_index('hnswlib.bin')

### Search

In [23]:
# load index
p = hnswlib.Index(space='l2', dim=dim)

p.load_index('hnswlib.bin', max_elements=num_elements)

In [24]:
# Controlling the recall by setting ef:
p.set_ef(50) # ef should always be > k

In [25]:
%%time

# search
labels, distances = p.knn_query(embs.iloc[542].values, k=10)

CPU times: user 1.74 ms, sys: 0 ns, total: 1.74 ms
Wall time: 1.25 ms


In [26]:
# labels and actual words
print(labels)
print([embmap[i] for i in labels.tolist()[0]])

[[   542  38654   1265    704 137015  13218  19003   1513   1318   5084]]
['software', 'softwares', 'Software', 'computer', 'sofware', 'freeware', 'shareware', 'tool', 'tools', 'enterprise']
